## Percepetrons as logical operators

In [2]:
test_inputs

[(0, 0), (0, 1), (1, 0), (1, 1)]

In [4]:
import pandas as pd

# TODO: Set weight1, weight2, and bias
weight1 = 1
weight2 = 1.0
bias = -2.0


# DON'T CHANGE ANYTHING BELOW
# Inputs and outputs
test_inputs = [(0, 0), (0, 1), (1, 0), (1, 1)]
correct_outputs = [False, False, False, True]
outputs = []

# Generate and check output
for test_input, correct_output in zip(test_inputs, correct_outputs):
    linear_combination = weight1 * test_input[0] + weight2 * test_input[1] + bias
    output = int(linear_combination >= 0)
    is_correct_string = 'Yes' if output == correct_output else 'No'
    outputs.append([test_input[0], test_input[1], linear_combination, output, is_correct_string])

# Print output
num_wrong = len([output[4] for output in outputs if output[4] == 'No'])
output_frame = pd.DataFrame(outputs, columns=['Input 1', '  Input 2', '  Linear Combination', '  Activation Output', '  Is Correct'])
if not num_wrong:
    print('Nice!  You got it all correct.\n')
else:
    print('You got {} wrong.  Keep trying!\n'.format(num_wrong))
print(output_frame.to_string(index=False))


Nice!  You got it all correct.

 Input 1    Input 2    Linear Combination    Activation Output   Is Correct
       0          0                  -2.0                    0          Yes
       0          1                  -1.0                    0          Yes
       1          0                  -1.0                    0          Yes
       1          1                   0.0                    1          Yes


In [14]:
w = -1
b = 1

In [15]:
x = 1
print('x=',x,'-> y=',w*x+b )

x = 0
print('x=',x,'-> y=',w*x+b )


x= 1 -> y= 0
x= 0 -> y= 1


## Gradient Calculations

Inspiration: https://github.com/yang-zhang/deep-learning

### Basic example: y = scalar, x = scalar

In [9]:
import torch
from torch.autograd import Variable
from torch import tensor

In [56]:
x = tensor(2.0, requires_grad=True)
y = 3*x**2
print('     x = ', x)
print('     y = ', y)
print('x.grad = ', x.grad)
print('\nAfter calling y.backward() once: we calculate the grad of y w.r.t. x, and assign this value to x.grad\n')
y.backward()
print('     x = ', x)
print('     y = ', y)
print('x.grad = ', x.grad)
print('   6*x = ', 6*x)

     x =  tensor(2., requires_grad=True)
     y =  tensor(12., grad_fn=<MulBackward0>)
x.grad =  None

After calling y.backward() once: we calculate the grad of y w.r.t. x, and assign this value to x.grad

     x =  tensor(2., requires_grad=True)
     y =  tensor(12., grad_fn=<MulBackward0>)
x.grad =  tensor(12.)
   6*x =  tensor(12., grad_fn=<MulBackward0>)


**Accumulating gradients** 

If now we create another variable as a function of x, and calculate the grad...we will notice that the grad does not match the derivative. Reason for this, is that x.grad accumulates the gradients. 

In [57]:
#x = tensor(2.0, requires_grad=True)
print('x.grad = ', x.grad)
previous_grad = x.grad.clone()
z = 4*x**2

print('\nCalling z.backward():')
z.backward()
print('     x = ', x)
print('     z = ', z)

print('   8*x = ', 8*x)
print('x.grad = ', x.grad)
print('8*x + previous_grad = ', 8*x + previous_grad)

x.grad =  tensor(12.)

Calling z.backward():
     x =  tensor(2., requires_grad=True)
     z =  tensor(16., grad_fn=<MulBackward0>)
   8*x =  tensor(16., grad_fn=<MulBackward0>)
x.grad =  tensor(28.)
8*x + previous_grad =  tensor(28., grad_fn=<AddBackward0>)


We can address this by setting x.grad = None, or by calling zero_grad() while training a model

In [59]:
x.grad = None
print('x.grad = ', x.grad)

x.grad =  None


### y = vector, x = vector

Output grad = vector

In [76]:
x = torch.rand(2, 1).requires_grad_(True)
y = x[0]**3 + 2*x[1]**2

print('x:\n ', x)
print('\ny= x[0]**3 + 2*x[1]**2:\n ', y)
print('\nx.grad:\n ', x.grad)

print('\n---\nAfter calling y.backward() once: we calculate the grad of y w.r.t. x, and assign this value to x.grad\n')
y.backward(gradient=torch.ones(1))
print('x:\n ', x)
print('y:\n ', y)
print('\nx.grad:\n ', x.grad)
print('\n3*x[0]**2, 4*x[1]:\n ', 3*x[0]**2,'\n', 4*x[1])

x:
  tensor([[0.2873],
        [0.3655]], requires_grad=True)

y= x[0]**3 + 2*x[1]**2:
  tensor([0.2910], grad_fn=<AddBackward0>)

x.grad:
  None

---
After calling y.backward() once: we calculate the grad of y w.r.t. x, and assign this value to x.grad

x:
  tensor([[0.2873],
        [0.3655]], requires_grad=True)
y:
  tensor([0.2910], grad_fn=<AddBackward0>)

x.grad:
  tensor([[0.2477],
        [1.4622]])

3*x[0]**2, 4*x[1]:
  tensor([0.2477], grad_fn=<MulBackward0>) 
 tensor([1.4622], grad_fn=<MulBackward0>)


### x = scalar, y = vector

In [78]:
x = Variable(torch.rand(1), requires_grad=True)
y = Variable(torch.zeros(2, 1), requires_grad=False)
y[0] = x**3
y[1] = 2*x**2

print('x:\n ', x)
print('\n y[0] = x**3')
print('\n y[1] = 2*x**2')
print('\nx.grad:\n ', x.grad)

print('\n---\nAfter calling y.backward() once: we calculate the grad of y w.r.t. x, and assign this value to x.grad\n')
y.backward(gradient=torch.ones(y.size()))
print('x:\n ', x)
print('y:\n ', y)
print('\nx.grad:\n ', x.grad)
print('\n3*x**2 + 4*x:\n ', 3*x**2 + 4*x)

x:
  tensor([0.4187], requires_grad=True)

 y[0] = x**3

 y[1] = 2*x**2

x.grad:
  None

---
After calling y.backward() once: we calculate the grad of y w.r.t. x, and assign this value to x.grad

x:
  tensor([0.4187], requires_grad=True)
y:
  tensor([[0.0734],
        [0.3505]], grad_fn=<CopySlices>)

x.grad:
  tensor([2.2004])

3*x**2 + 4*x:
  tensor([2.2004], grad_fn=<AddBackward0>)


## Data Loaders